In [1]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [2]:
import sys

import pandas as pd
import numpy as np
import time
import datetime
import os
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper

from nanoHUB.clustering.infra import get_mike_only_clustered_users, get_xufeng_only_clustered_users, get_mike_xufeng_clustered_users
from nanoHUB.clustering.infra import create_clusters_repository, add_cluster_info, get_all_clustered_users, get_all_self_study_users, get_all_self_identified_users
from nanoHUB.dataaccess.lake import new_session_lifetime_calculator

application = Application.get_instance('DEBUG')

nanoHUB - Serving Students, Researchers & Instructors


In [3]:
pd.set_option('display.max_columns', None)
cwd = os.getcwd()

In [4]:
## determine starting year
#if cache_flag == False:
start_year_base = 1999 #2000
start_year_base = 1999 #2000
pathsave=cwd+'/cached_data/'
pathsave=cwd+'/test_data/'
pathsave=cwd+'/SiteVisit2022_data/'

lifetime_cache = True

#else:
#     print('check the saved file')
#    start_year_base = int(user_breakdown_df['year_month'].to_list()[-1][:4])
    
    

In [5]:
if (lifetime_cache == False):
    date_difference = new_session_lifetime_calculator(application)


In [6]:
if (lifetime_cache == False):
    day_1_users_df = date_difference.get_24_hour_users()

    print("Total one day users: %d" % len(day_1_users_df))
    #display(day_1_users_df.head())
    #display(day_1_users_df.tail())
    day_1_users = day_1_users_df['user'].tolist()
    #display(day_1_users[399:420])

In [7]:
if (lifetime_cache == False):
    day_7_users_df = date_difference.get_7_day_users()

    print("Total 7 day users: %d" % len(day_7_users_df))
    #display(day_7_users_df.head())
    #display(day_7_users_df.tail())
    day_7_users = day_7_users_df['user'].tolist()

if (lifetime_cache == False):
    print("calculate a subset of users that excludes the previous subset")
    print('day_1_users length = ',len(day_1_users))
    print("day_7_users length = ",len(day_7_users))
    day_2_7_users = [x for x in day_7_users if x  not in day_1_users]
    print('day_2_7_users length = ',len(day_2_7_users))


In [8]:
if (lifetime_cache == False):
    day_30_users_df = date_difference.get_30_day_users()

    print("Total 30 day users: %d" % len(day_30_users_df))
    display(day_30_users_df.head())
    display(day_30_users_df.tail())
    day_30_users = day_30_users_df['user'].tolist()

if (lifetime_cache == False):
    print("calculate a subset of users that excludes the previous subset")
    print('day_7_users length = ',len(day_7_users))
    print("day_30_users length = ",len(day_30_users))
    day_8_30_users = [x for x in day_30_users if x  not in day_7_users]
    print('day_8_30_users length = ',len(day_8_30_users))

In [9]:
if (lifetime_cache == False):
    day_90_users_df = date_difference.get_90_day_users()

    print("Total 90 day users: %d" % len(day_90_users_df))
    display(day_90_users_df.head())
    display(day_90_users_df.tail())
    day_90_users = day_90_users_df['user'].tolist()

if (lifetime_cache == False):
    print("calculate a subset of users that excludes the previous subset")
    print('day_30_users length = ',len(day_30_users))
    print("day_90_users length = ",len(day_90_users))
    day_31_90_users = [x for x in day_90_users if x  not in day_30_users]
    print('day_31_90_users length = ',len(day_31_90_users))

In [10]:
if (lifetime_cache == False):
    day_180_users_df = date_difference.get_180_day_users()

    print("Total 180 day users: %d" % len(day_180_users_df))
    display(day_180_users_df.head())
    display(day_180_users_df.tail())
    day_180_users = day_180_users_df['user'].tolist()

if (lifetime_cache == False):
    print("calculate a subset of users that excludes the previous subset")
    print('day_90_users length = ',len(day_90_users))
    print("day_180_users length = ",len(day_180_users))
    day_91_180_users = [x for x in day_180_users if x  not in day_90_users]
    print('day_91_180_users length = ',len(day_91_180_users))

In [11]:
if (lifetime_cache == False):
    day_365_users_df = date_difference.get_365_day_users()

    print("Total one year users: %d" % len(day_365_users_df))
    display(day_365_users_df.head())
    display(day_365_users_df.tail())

    day_365_users = day_365_users_df['user'].tolist()

if (lifetime_cache == False):
    print("calculate a subset of users that excludes the previous subset")
    print('day_180_users length = ',len(day_180_users))
    print("day_365_users length = ",len(day_365_users))
    day_181_365_users = [x for x in day_365_users if x  not in day_180_users]
    print('day_181_365_users length = ',len(day_181_365_users))


In [12]:
if (lifetime_cache == False):
    years_users_df = date_difference.get_more_than_1_year_users()

    print("Total more than 1 year users: %d" % len(years_users_df))
    display(years_users_df.head())
    display(years_users_df.tail())
    years_users = years_users_df['user'].tolist()

In [13]:
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('rfm_data')

s3_client = get_default_s3_client(application)
derived_data_file_path = 'derived_data_for_users.csv'
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

In [14]:
def Annualized_Usage(nanohub_metrics_db,start_year,list_usernames,list_title,pathsave):
    today = datetime.datetime.today()
    print(today)    
    #cache_r_track = pd.DataFrame()
    #cache_si_track = pd.DataFrame()
    #cache_si_track['user'] = self_identified_track
    active_track = []
    active_date = []
    
    ## computer has memory limits, so split toolstart into branches
    # this filters and finds the research usage in nanohub
    while start_year < today.year+1:
        start_year += 1
        end_date = r"'"+str(start_year)+r"-01-01'"
        start_date = r"'"+str(start_year-2)+r"-01-01'"

        print('end date: '+ str(end_date))
        for i in range(1,13):
            if i < 10:
                start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
                end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
            else:
                start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
                end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

            sql_query = "select user from toolstart where user in " + str(tuple(list_usernames))\
                +" and datetime <= "+end_date+" and datetime >= "+start_date
            active_users = pd.read_sql_query(sql_query, nanohub_metrics_db)
            #print('date '+str(end_date)+' '+list_title+' partial',active_users.drop_duplicates()[0:10])
            print('date '+str(end_date)+' '+list_title+' length',len(active_users.drop_duplicates()))

            #count the number of active users and append into an array
            active_track.append(active_users.drop_duplicates().shape[0])
            active_date.append(end_date)


        print('end date: '+ str(end_date))
        #display(researcher_track)
        #display(researcher_track)

        cache_track = pd.DataFrame()
        cache_track['track'] = active_track
        cache_track.to_csv(pathsave+list_title+'_track.csv')
        cache_track.to_csv(pathsave+ str(start_year)+'_'+list_title+'_track.csv')
        cache_track = []
        
        cache_datetrack = pd.DataFrame()
        cache_datetrack['date'] = active_date
        cache_datetrack['track'] = active_track
        cache_datetrack.to_csv(pathsave+list_title+'_date_track.csv')
        cache_datetrack.to_csv(pathsave+ str(start_year)+'_'+list_title+'_date_track.csv')
        cache_datetrack = []
        

    return active_track

## Cumulative classroom/research/self-study

In [15]:
clustered_df = get_all_clustered_users(processed_mapper, derived_data_file_path)
clustered_df['nanoHUB_user_ID__c'] = clustered_df['id']
clustered_NH_ids = clustered_df['nanoHUB_user_ID__c']
display(len(clustered_NH_ids))

79465

In [16]:
mike_clustered_users_df = get_mike_only_clustered_users(processed_mapper, derived_data_file_path)
mike_clustered_users_df['nanoHUB_user_ID__c'] = mike_clustered_users_df['id']
mike_clustered_NH_ids = mike_clustered_users_df['nanoHUB_user_ID__c']
display(len(mike_clustered_users_df))

[WARNING] [warnings - py.warnings]: /home/gekco/nanoHUB/nanoHUB/clustering/infra.py:389: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return df[df[column_name].str.contains(search_for)]
 [warnings._showwarnmsg:109]


17004

In [17]:
xufeng_clustered_users_df = get_xufeng_only_clustered_users(processed_mapper, derived_data_file_path)
xufeng_clustered_users_df['nanoHUB_user_ID__c'] = xufeng_clustered_users_df['id']
xufeng_clustered_NH_ids = xufeng_clustered_users_df['nanoHUB_user_ID__c']
display(len(xufeng_clustered_users_df))

15270

In [18]:
mike_xufeng_clustered_users_df = get_mike_xufeng_clustered_users(processed_mapper, derived_data_file_path)
mike_xufeng_clustered_users_df['nanoHUB_user_ID__c'] = mike_xufeng_clustered_users_df['id']
mike_xufeng_clustered_NH_ids = mike_xufeng_clustered_users_df['nanoHUB_user_ID__c']
display(len(mike_xufeng_clustered_users_df))

47191

In [19]:
# raise

In [20]:
self_study_df = get_all_self_study_users(processed_mapper, derived_data_file_path)
self_study_df['nanoHUB_user_ID__c'] = self_study_df['id']
self_study_NH_ids = self_study_df['nanoHUB_user_ID__c']
self_study_usernames = self_study_df['username'].tolist()

print('self_study_NH_ids length = ',len(self_study_NH_ids))
print('self_study_usernames length = ',len(self_study_usernames))
print('self_study_usernames samples = ',self_study_usernames[0:10])

self_study_NH_ids length =  193226
self_study_usernames length =  193226
self_study_usernames samples =  ['hubrepo', 'admin', 'support', 'gridstat', 'ncn', 'nanohub', 'repo', 'violin', 'mireles', 'k_dadesh']


In [21]:
# self_identified users, are individuals that have a researcherID, google scholar etc in their profile
self_identified_df = get_all_self_identified_users(processed_mapper, derived_data_file_path)
self_identified_df['nanoHUB_user_ID__c'] = self_identified_df['id']
self_identified_NH_ids = self_identified_df['nanoHUB_user_ID__c']
self_identified_usernames = self_identified_df['username'].tolist()
print('length of self_identified_NH_ids',len(self_identified_NH_ids))
display(self_identified_df.head())



length of self_identified_NH_ids 4037


,id,name,username,email,registerDate,lastvisitDate,number_simulations,uid,profile_key,has_researcher_profile,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,mike_spring_2021,xufeng_spring_2021,nanoHUB_user_ID__c
895,2579,Stefan Birner,birner,stefan.birner@nextnano.com,2006-05-05 07:36:09,2020-01-13 15:58:33,52.0,2579.0,[orcid],True,['mike_spring_2011'],False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2579
1176,2862,Mark Lundstrom,lundstro,lundstro@purdue.edu,2004-12-06 22:01:25,2022-02-08 16:28:36,2345.0,2862.0,"[googlescholar, researchgateid]",True,"['xufeng_fall_2008', 'xufeng_spring_2009', 'xu...",False,True,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,2862
1473,3159,Tamara J. Moore,tmoore,tmoore@purdue.edu,2003-09-03 21:18:50,2021-10-07 19:58:59,38.0,3159.0,[orcid],True,['xufeng_fall_2020'],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,3159
1796,3482,Gerhard Klimeck,gekco,gekco@purdue.edu,2004-04-10 17:58:53,2022-02-10 14:45:37,18677.0,3482.0,"[orcid, googlescholar, researcherid, researchg...",True,"['xufeng_fall_2009', 'xufeng_fall_2010', 'xufe...",False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,3482
1897,3583,Eric Pop,epop,epop@stanford.edu,2006-02-09 20:57:14,2021-09-04 07:19:40,90.0,3583.0,"[orcid, googlescholar]",True,['xufeng_fall_2012'],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3583


In [22]:
allusers_df = processed_mapper.read(derived_data_file_path)
display(allusers_df.head(5))
#display(allusers_usernames)
# we know there are some users who have a NULL user name - they need to be taken out
display(allusers_df.loc[allusers_df['username'].isnull()])
#display(allusers_df.loc[allusers_df['username'].notna()])

# take out the users with a NULL user name
allusers_df = allusers_df.loc[allusers_df['username'].notna()]
display(allusers_df.loc[allusers_df['username'].isnull()])

# create a list of all user names
allusers_usernames = allusers_df['username'].tolist()

,id,name,username,email,registerDate,lastvisitDate,number_simulations,uid,profile_key,has_researcher_profile,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,mike_spring_2021,xufeng_spring_2021
0,998,hubrepo hubrepo,hubrepo,nkissebe@gmail.com,2014-11-13 21:09:09,NaN,0.0,NaN,[],False,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1000,CMS Admin Manager,admin,support@nanohub.org,NaN,NaN,0.0,NaN,[],False,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1683,nanoHUB support,support,support@nanohub.org,2008-11-19 22:51:04,2008-11-19 23:55:30,2.0,1683.0,[],False,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1684,Grid Statistics,gridstat,gridstat@nanohub.org,2008-11-18 17:29:56,2020-02-14 18:50:14,0.0,NaN,[],False,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,1685,NCN NCN,ncn,ncn@nanohub.org,2008-11-11 19:17:04,NaN,0.0,NaN,[],False,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


,id,name,username,email,registerDate,lastvisitDate,number_simulations,uid,profile_key,has_researcher_profile,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,mike_spring_2021,xufeng_spring_2021
77305,82487,Geraldine Young,NaN,f948941@rmqkr.net,2013-05-22 20:04:55,2013-06-14 20:51:07,39.0,82487.0,[],False,"['mike_fall_2008', 'xufeng_fall_2008', 'mike_s...",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
211599,281303,Nataly Cárdenas,NaN,njcardenasvasquez@gmail.com,2020-03-20 17:26:07,2020-03-20 17:26:07,4.0,281303.0,[],False,"['mike_fall_2008', 'xufeng_fall_2008', 'mike_s...",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


,id,name,username,email,registerDate,lastvisitDate,number_simulations,uid,profile_key,has_researcher_profile,clusters,mike_fall_2008,xufeng_fall_2008,mike_spring_2008,xufeng_spring_2008,mike_fall_2009,xufeng_fall_2009,mike_spring_2009,xufeng_spring_2009,mike_fall_2010,xufeng_fall_2010,mike_spring_2010,xufeng_spring_2010,mike_fall_2011,xufeng_fall_2011,mike_spring_2011,xufeng_spring_2011,mike_fall_2012,xufeng_fall_2012,mike_spring_2012,xufeng_spring_2012,mike_fall_2013,xufeng_fall_2013,mike_spring_2013,xufeng_spring_2013,mike_fall_2014,xufeng_fall_2014,mike_spring_2014,xufeng_spring_2014,mike_fall_2015,xufeng_fall_2015,mike_spring_2015,xufeng_spring_2015,mike_fall_2016,xufeng_fall_2016,mike_spring_2016,xufeng_spring_2016,mike_fall_2017,xufeng_fall_2017,mike_spring_2017,xufeng_spring_2017,mike_fall_2018,xufeng_fall_2018,mike_spring_2018,xufeng_spring_2018,mike_fall_2019,xufeng_fall_2019,mike_spring_2019,xufeng_spring_2019,mike_fall_2020,xufeng_fall_2020,mike_spring_2020,xufeng_spring_2020,mike_fall_2021,xufeng_fall_2021,mike_spring_2021,xufeng_spring_2021


In [23]:
# ideally we would not search activities for all users but only in the active users.
# so here we could determine the list of all active users to make the queries for total sim users over time 
# run faster
# 

#sql_query_simtotal = "select distinct user from toolstart;"
#simtotal_users = pd.read_sql_query(sql_query_simtotal, nanohub_metrics_db) #engine_metrics #.drop_duplicates()


In [24]:
#print('simtotal_users length', len(simtotal_users))
#display(simtotal_users)


In [25]:
print('clustered users',clustered_NH_ids.shape)
print('selfstudy users',self_study_NH_ids.shape)
print('self_study_usernames',len(self_study_usernames))
print('self identified researchers',self_identified_NH_ids.shape)
print('all users',allusers_df.shape)

clustered users (79465,)
selfstudy users (193226,)
self_study_usernames 193226
self identified researchers (4037,)
all users (272689, 67)


In [26]:
frames = [clustered_df, self_study_df]
contacts_df = pd.concat(frames)

In [27]:
## need to filter self-study-nh-ids with the researchers

In [28]:
## This researcher determination is based entirely on the results of research authors on nanohub
## i.e., we use the researcher information on db2

In [29]:
sql_query = 'select id, uid, author from jos_citations'
jos_citations = pd.read_sql_query(sql_query, nanohub_db)
display(jos_citations.head(2))
print(jos_citations.shape)

,id,uid,author
0,10000001,5568,"Weber, Bent; Mahapatra, Suddhasatta; Ryu, Hoon..."
1,10000002,5568,"Andrawis, Robert; Bermeo, Jose; Charles, James..."


(4203, 3)


In [30]:
t1_authors = jos_citations['author'].to_list()
t1_authors2 = [] #[j.split(';') for i,j in enumerate(t1_authors)]

In [31]:
for i,j in enumerate(t1_authors):
    try:
        holder = j.split(';')
        if len(holder) > 1: 
            for k,l in enumerate(holder):
                if l[0] == ' ':
                    holder[k] = l[1:]
        t1_authors2.append(holder)
    except:
        garb = True

In [32]:
t1_authors32 = [item for sublist in t1_authors2 for item in sublist]
t1_authors3 = set(t1_authors32)
print(len(t1_authors3))

6860


In [33]:
## extract all researchers with nh ids
rw_ids = []
need_search = []
for i,j in enumerate(t1_authors3):
    if '{{' in j:
        rw_ids.append(i)
    else:
        need_search.append(i)

In [34]:
## from the rw_ids, extract the nanohub userids
# can obtain usernames from the DB2 table
rw_ids2 = np.array(list(t1_authors3))[rw_ids]
for i,j in enumerate(rw_ids2):
    begin = j.index('{{')
    rw_ids2[i] = j[begin+2:-2]
print(len(rw_ids2))

2079


In [35]:
# need to obtain usernames
sql_query = "select id,name,username from jos_users where id in "+str(tuple(rw_ids2))
rw_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(rw_researchers.head(2))

,id,name,username
0,1742,H.-S. Philip Wong,hspwong
1,1767,Osama Munir Nayfeh,onayfeh


In [36]:
## need to obtain the username/ids from those people without nh ids explicitly spec'd in jos_citations
ns2 = np.array(list(t1_authors3))[need_search]
ns2 = ns2[1:]

In [37]:
tuple(ns2)[:5]

('Michel Dubois',
 'CA Achete',
 'Thomas Sterling',
 'Sangtae Kim',
 'Y.-R. Chiang')

In [38]:
sql_query = "select id,name,username from jos_users where name in "+str(tuple(ns2))
ns2_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(ns2_researchers.head(2))

,id,name,username
0,282167,Aarthi Narayanan,aarthin0102
1,280280,Abani Patra,apatra01


In [39]:
## combine the usernames into one data structure
researcher_usernames = rw_researchers['username'].to_list() + ns2_researchers['username'].to_list()
print('list of researchers who have cited nanohub  researcher_usernames=',len(researcher_usernames))

list of researchers who have cited nanohub  researcher_usernames= 3046


In [40]:
#need to take out the researchers and citing authors from the list of selfstudy users. 
#
print('self_identified_usernames can have overlap with citing or self identified researchers',len(self_identified_usernames))
print('take out overlap of len(researcher_usernames)',len(researcher_usernames))
print('take from len(self_identified_usernames)',len(self_identified_usernames))

print('number of self_identified_usernames users start',len(self_study_usernames))
self_identified_usernames = [x for x in self_identified_usernames if x not in researcher_usernames]
print('number of self_identified_usernames users end',len(self_study_usernames))

print('number of self-study users self_study_usernames with research',len(self_study_usernames))
self_study_usernames = [x for x in self_study_usernames if x not in researcher_usernames]
print('number of self-study users self_study_usernames without research',len(self_study_usernames))

print('number of self-study users self_study_usernames with self identified research',len(self_study_usernames))
self_study_usernames = [x for x in self_study_usernames if x not in self_identified_usernames]
print('number of self-study users self_study_usernames without self identified research',len(self_study_usernames))

self_identified_usernames can have overlap with citing or self identified researchers 4037
take out overlap of len(researcher_usernames) 3046
take from len(self_identified_usernames) 4037
number of self_identified_usernames users start 193226
number of self_identified_usernames users end 193226
number of self-study users self_study_usernames with research 193226
number of self-study users self_study_usernames without research 191201
number of self-study users self_study_usernames with self identified research 191201
number of self-study users self_study_usernames without self identified research 187882


In [41]:
if (lifetime_cache == False):
    print("separate the self study users by lifetime")
    print('self_study_NH_ids length = ',len(self_study_NH_ids))
    print("total day_1_users length = ",len(day_1_users))
    self_study_day_1_users = [x for x in day_1_users  if x  in self_study_usernames]
    print('number of self_study_day_1_users',len(self_study_day_1_users))

    print("total day_2_7_users length = ",len(day_2_7_users))
    self_study_day_2_7_users = [x for x in day_2_7_users  if x  in self_study_usernames]
    print('number of self_study_day_2_7_users',len(self_study_day_2_7_users))

    print("total day_8_30_users length = ",len(day_8_30_users))
    self_study_day_8_30_users = [x for x in day_8_30_users  if x  in self_study_usernames]
    print('number of self_study_day_8_30_users',len(self_study_day_8_30_users))

    print("total day_31_90_users length = ",len(day_31_90_users))
    self_study_day_31_90_users = [x for x in day_31_90_users  if x  in self_study_usernames]
    print('number of self_study_day_31_90_users',len(self_study_day_31_90_users))

    print("total day_91_180_users length = ",len(day_91_180_users))
    self_study_day_91_180_users = [x for x in day_91_180_users  if x  in self_study_usernames]
    print('number of self_study_day_91_180_users',len(self_study_day_91_180_users))

    print("total day_181_365_users length = ",len(day_181_365_users))
    self_study_day_181_365_users = [x for x in day_181_365_users  if x  in self_study_usernames]
    print('number of self_study_day_181_365_users',len(self_study_day_181_365_users))


    print("total years_users length = ",len(years_users))
    self_study_years_users = [x for x in years_users  if x  in self_study_usernames]
    print('number of self_study_years_users',len(self_study_years_users))

In [42]:
today = datetime.datetime.today()
print(today)

2022-03-04 17:24:30.990184


In [43]:
print(pathsave)


/home/gekco/nanoHUB/nanoHUB/pipeline/plotting/SiteVisit2022_data/


In [44]:
#print(user_breakdown_df['year_month'].to_list()[-1][:4])

In [45]:
from copy import deepcopy

In [46]:
all_months_list = list()
cache_flag = False
for this_year in range(start_year_base+1,today.year+2):
    if cache_flag == True:
        for this_month in range(1,today.month+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))
    else:
        for this_month in range(1,12+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))

cache_all_month_list = pd.DataFrame()
cache_all_month_list['Date'] = all_months_list
cache_all_month_list.to_csv(pathsave+'all_months_list.csv')
cache_all_month_list.to_csv(pathsave+ str(start_year_base)+'_all_months_list.csv')
cache_all_month_list = []
print('allmonths partial list',all_months_list[0:36])


allmonths partial list [datetime.datetime(2000, 1, 1, 0, 0), datetime.datetime(2000, 2, 1, 0, 0), datetime.datetime(2000, 3, 1, 0, 0), datetime.datetime(2000, 4, 1, 0, 0), datetime.datetime(2000, 5, 1, 0, 0), datetime.datetime(2000, 6, 1, 0, 0), datetime.datetime(2000, 7, 1, 0, 0), datetime.datetime(2000, 8, 1, 0, 0), datetime.datetime(2000, 9, 1, 0, 0), datetime.datetime(2000, 10, 1, 0, 0), datetime.datetime(2000, 11, 1, 0, 0), datetime.datetime(2000, 12, 1, 0, 0), datetime.datetime(2001, 1, 1, 0, 0), datetime.datetime(2001, 2, 1, 0, 0), datetime.datetime(2001, 3, 1, 0, 0), datetime.datetime(2001, 4, 1, 0, 0), datetime.datetime(2001, 5, 1, 0, 0), datetime.datetime(2001, 6, 1, 0, 0), datetime.datetime(2001, 7, 1, 0, 0), datetime.datetime(2001, 8, 1, 0, 0), datetime.datetime(2001, 9, 1, 0, 0), datetime.datetime(2001, 10, 1, 0, 0), datetime.datetime(2001, 11, 1, 0, 0), datetime.datetime(2001, 12, 1, 0, 0), datetime.datetime(2002, 1, 1, 0, 0), datetime.datetime(2002, 2, 1, 0, 0), datetime

In [47]:
# clustered user usernames
tc_query = 'select id,username from jos_users where id in ' +str(tuple(clustered_NH_ids))
clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(mike_clustered_NH_ids))
mike_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(mike_clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(xufeng_clustered_NH_ids))
xufeng_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(xufeng_clustered_NH_username))

tc_query = 'select id,username from jos_users where id in ' +str(tuple(mike_xufeng_clustered_NH_ids))
mike_xufeng_clustered_NH_username = pd.read_sql_query(tc_query,nanohub_db)
display(len(mike_xufeng_clustered_NH_username))

79465

17004

15270

47191

In [48]:
if (lifetime_cache == False):
    clustered_all = clustered_NH_username['username'].tolist()
    print("separate the cluster users by lifetime")
    print('clustered_all length = ',len(clustered_all))
    print("total day_1_users length = ",len(day_1_users))
    clustered_all_day_1_users = [x for x in day_1_users  if x  in clustered_all]
    print('number of clustered_all_day_1_users',len(clustered_all_day_1_users))

    print("total day_2_7_users length = ",len(day_2_7_users))
    clustered_all_day_2_7_users = [x for x in day_2_7_users  if x  in clustered_all]
    print('number of clustered_all_day_2_7_users',len(clustered_all_day_2_7_users))

    print("total day_8_30_users length = ",len(day_8_30_users))
    clustered_all_day_8_30_users = [x for x in day_8_30_users  if x  in clustered_all]
    print('number of clustered_all_day_8_30_users',len(clustered_all_day_8_30_users))

    print("total day_31_90_users length = ",len(day_31_90_users))
    clustered_all_day_31_90_users = [x for x in day_31_90_users  if x  in clustered_all]
    print('number of clustered_all_day_31_90_users',len(clustered_all_day_31_90_users))

    print("total day_91_180_users length = ",len(day_91_180_users))
    clustered_all_day_91_180_users = [x for x in day_91_180_users  if x  in clustered_all]
    print('number of clustered_all_day_91_180_users',len(clustered_all_day_91_180_users))

    print("total day_181_365_users length = ",len(day_181_365_users))
    clustered_all_day_181_365_users = [x for x in day_181_365_users  if x  in clustered_all]
    print('number of clustered_all_day_181_365_users',len(clustered_all_day_181_365_users))


    print("total years_users length = ",len(years_users))
    clustered_all_years_users = [x for x in years_users  if x  in clustered_all]
    print('number of clustered_ally_years_users',len(clustered_all_years_users))

In [49]:
if lifetime_cache == True:
    # read files
    print("need to read files here")
    read_df = pd.read_csv(pathsave+'day_1_users.csv',usecols=['username'])
    day_1_users = read_df['username'].tolist()
    print('day_1_users length =',len(day_1_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'day_2_7_users.csv',usecols=['username'])
    day_2_7_users = read_df['username'].tolist()
    print('day_2_7_users length =',len(day_2_7_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'day_8_30_users.csv',usecols=['username'])
    day_8_30_users = read_df['username'].tolist()
    print('day_8_30_users length =',len(day_8_30_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'day_31_90_users.csv',usecols=['username'])
    day_31_90_users = read_df['username'].tolist()
    print('day_31_90_users length =',len(day_31_90_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'day_91_180_users.csv',usecols=['username'])
    day_91_180_users = read_df['username'].tolist()
    print('day_91_180_users length =',len(day_91_180_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'day_181_365_users.csv',usecols=['username'])
    day_181_365_users = read_df['username'].tolist()
    print('day_181_365_users length =',len(day_181_365_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'years_users.csv',usecols=['username'])
    years_users = read_df['username'].tolist()
    print('years_users length =',len(years_users))
    read_df = []

    read_df = pd.read_csv(pathsave+'self_study_day_1_users.csv',usecols=['username'])
    self_study_day_1_users = read_df['username'].tolist()
    print('self_study_day_1_users length =',len(self_study_day_1_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_day_2_7_users.csv',usecols=['username'])
    self_study_day_2_7_users = read_df['username'].tolist()
    print('self_study_day_2_7_users length =',len(self_study_day_2_7_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_day_8_30_users.csv',usecols=['username'])
    self_study_day_8_30_users = read_df['username'].tolist()
    print('self_study_day_8_30_users length =',len(self_study_day_8_30_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_day_31_90_users.csv',usecols=['username'])
    self_study_day_31_90_users = read_df['username'].tolist()
    print('self_study_day_31_90_users length =',len(self_study_day_31_90_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_day_91_180_users.csv',usecols=['username'])
    self_study_day_91_180_users = read_df['username'].tolist()
    print('self_study_day_91_180_users length =',len(self_study_day_91_180_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_day_181_365_users.csv',usecols=['username'])
    self_study_day_181_365_users = read_df['username'].tolist()
    print('self_study_day_181_365_users length =',len(self_study_day_181_365_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'self_study_years_users.csv',usecols=['username'])
    self_study_years_users = read_df['username'].tolist()
    print('self_study_years_users length =',len(self_study_years_users))
    read_df = []

    read_df = pd.read_csv(pathsave+'clustered_all_day_1_users.csv',usecols=['username'])
    clustered_all_day_1_users = read_df['username'].tolist()
    print('clustered_all_day_1_users length =',len(clustered_all_day_1_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_day_2_7_users.csv',usecols=['username'])
    clustered_all_day_2_7_users = read_df['username'].tolist()
    print('clustered_all_day_2_7_users length =',len(clustered_all_day_2_7_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_day_8_30_users.csv',usecols=['username'])
    clustered_all_day_8_30_users = read_df['username'].tolist()
    print('clustered_all_day_8_30_users length =',len(clustered_all_day_8_30_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_day_31_90_users.csv',usecols=['username'])
    clustered_all_day_31_90_users = read_df['username'].tolist()
    print('clustered_all_day_31_90_users length =',len(clustered_all_day_31_90_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_day_91_180_users.csv',usecols=['username'])
    clustered_all_day_91_180_users = read_df['username'].tolist()
    print('clustered_all_day_91_180_users length =',len(clustered_all_day_91_180_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_day_181_365_users.csv',usecols=['username'])
    clustered_all_day_181_365_users = read_df['username'].tolist()
    print('clustered_all_day_181_365_users length =',len(clustered_all_day_181_365_users))
    read_df = []
    read_df = pd.read_csv(pathsave+'clustered_all_years_users.csv',usecols=['username'])
    clustered_all_years_users = read_df['username'].tolist()
    print('clustered_all_years_users length =',len(clustered_all_years_users))
    read_df = []

 
else:
    #write the computed data into a cache
    ## cache the researchers, self-study, and clustered users
    cache_write = pd.DataFrame()
    cache_write['username'] = day_1_users
    cache_write.to_csv(pathsave+'day_1_users.csv')
    cache_write = []
    
    cache_write = pd.DataFrame()
    cache_write['username'] = day_2_7_users
    cache_write.to_csv(pathsave+'day_2_7_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = day_8_30_users
    cache_write.to_csv(pathsave+'day_8_30_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = day_31_90_users
    cache_write.to_csv(pathsave+'day_31_90_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = day_91_180_users
    cache_write.to_csv(pathsave+'day_91_180_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = day_181_365_users
    cache_write.to_csv(pathsave+'day_181_365_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = years_users
    cache_write.to_csv(pathsave+'years_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_1_users
    cache_write.to_csv(pathsave+'self_study_day_1_users.csv')
    cache_write = []
    
    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_2_7_users
    cache_write.to_csv(pathsave+'self_study_day_2_7_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_8_30_users
    cache_write.to_csv(pathsave+'self_study_day_8_30_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_31_90_users
    cache_write.to_csv(pathsave+'self_study_day_31_90_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_91_180_users
    cache_write.to_csv(pathsave+'self_study_day_91_180_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_day_181_365_users
    cache_write.to_csv(pathsave+'self_study_day_181_365_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = self_study_years_users
    cache_write.to_csv(pathsave+'self_study_years_users.csv')
    cache_write = []
    
    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_1_users
    cache_write.to_csv(pathsave+'clustered_all_day_1_users.csv')
    cache_write = []
    
    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_2_7_users
    cache_write.to_csv(pathsave+'clustered_all_day_2_7_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_8_30_users
    cache_write.to_csv(pathsave+'clustered_all_day_8_30_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_31_90_users
    cache_write.to_csv(pathsave+'clustered_all_day_31_90_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_91_180_users
    cache_write.to_csv(pathsave+'clustered_all_day_91_180_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_day_181_365_users
    cache_write.to_csv(pathsave+'clustered_all_day_181_365_users.csv')
    cache_write = []

    cache_write = pd.DataFrame()
    cache_write['username'] = clustered_all_years_users
    cache_write.to_csv(pathsave+'clustered_all_years_users.csv')
    cache_write = []

need to read files here
day_1_users length = 73143
day_2_7_users length = 15596
day_8_30_users length = 20583
day_31_90_users length = 19621
day_91_180_users length = 8592
day_181_365_users length = 7251
years_users length = 18589
self_study_day_1_users length = 46283
self_study_day_2_7_users length = 5948
self_study_day_8_30_users length = 6462
self_study_day_31_90_users length = 6042
self_study_day_91_180_users length = 3244
self_study_day_181_365_users length = 3310
self_study_years_users length = 8861
clustered_all_day_1_users length = 25038
clustered_all_day_2_7_users length = 9367
clustered_all_day_8_30_users length = 13830
clustered_all_day_31_90_users length = 13293
clustered_all_day_91_180_users length = 5216
clustered_all_day_181_365_users length = 3772
clustered_all_years_users length = 8930


In [50]:
## self study user usernames
#ts_query = 'select id,username from jos_users where id in ' + str(tuple(self_study_NH_ids.dropna().to_list())) 
##sf_contact_ids['nanoHUB_user_ID__c']))
#ss_NH_username = pd.read_sql_query(ts_query,nanohub_db)

In [51]:
## cache the researchers, self-study, and clustered users
cache_r_nhid = pd.DataFrame()
#cache_r_nhid['nhid'] = researcher_ids
cache_r_nhid['username'] = researcher_usernames

cache_si_nhid = pd.DataFrame()
#cache_si_nhid['nhid'] = self_identified_NH_ids.to_list()
cache_si_nhid['username'] = self_identified_usernames

cache_ss_nhid = pd.DataFrame()
#cache_ss_nhid['nhid'] = ss_NH_username['id']
cache_ss_nhid['username'] = self_study_usernames

cache_c_nhid = pd.DataFrame()
cache_c_nhid['nhid'] = clustered_NH_username['id']
cache_c_nhid['username'] = clustered_NH_username['username']

cache_cm_nhid = pd.DataFrame()
cache_cm_nhid['nhid'] = mike_clustered_NH_username['id']
cache_cm_nhid['username'] = mike_clustered_NH_username['username']

cache_cx_nhid = pd.DataFrame()
cache_cx_nhid['nhid'] = xufeng_clustered_NH_username['id']
cache_cx_nhid['username'] = xufeng_clustered_NH_username['username']

cache_cmx_nhid = pd.DataFrame()
cache_cmx_nhid['nhid'] = mike_xufeng_clustered_NH_username['id']
cache_cmx_nhid['username'] = mike_xufeng_clustered_NH_username['username']

In [52]:
print(cache_r_nhid.head(2))
print(cache_si_nhid.head(2))
print(cache_ss_nhid.head(2))
print(cache_c_nhid.head(2))
print(cache_cm_nhid.head(2))
print(cache_cx_nhid.head(2))
print(cache_cmx_nhid.head(2))

  username
0  hspwong
1  onayfeh
  username
0   tmoore
1  alexwei
  username
0  hubrepo
1    admin
     nhid    username
0  166284  0.yao.yuan
1  344801  007sribabu
   nhid username
0  1767  onayfeh
1  1909  reimers
   nhid username
0  1848   kevdog
1  1852  himadri
   nhid      username
0  1794        mar889
1  2136  ashtone_rose


In [53]:
cache_r_nhid.to_csv(pathsave+'citingauthor_ids.csv')
cache_si_nhid.to_csv(pathsave+'selfidentifiedauthors_ids.csv')
cache_ss_nhid.to_csv(pathsave+'unclassified_ids.csv')
cache_c_nhid.to_csv(pathsave+'clustered_all_ids.csv')
cache_cm_nhid.to_csv(pathsave+'clustered_m_ids.csv')
cache_cx_nhid.to_csv(pathsave+'clustered_x_ids.csv')
cache_cmx_nhid.to_csv(pathsave+'clustered_mx_ids.csv')

In [55]:
#call the annual number analysis as a function
start_year = start_year_base
#clustered_track       = Annualized_Usage(nanohub_metrics_db,start_year,clustered_NH_username['username'],'clustered_all',pathsave)
#clustered_m_track     = Annualized_Usage(nanohub_metrics_db,start_year,mike_clustered_NH_username['username'],'clustered_m',pathsave)
#clustered_x_track     = Annualized_Usage(nanohub_metrics_db,start_year,xufeng_clustered_NH_username['username'],'clustered_x',pathsave)
#clustered_mx_track    = Annualized_Usage(nanohub_metrics_db,start_year,mike_xufeng_clustered_NH_username['username'],'clustered_mx',pathsave)
#researcher_track      = Annualized_Usage(nanohub_metrics_db,start_year,researcher_usernames,'citingauthor',pathsave)
#self_identified_track = Annualized_Usage(nanohub_metrics_db,start_year,self_identified_usernames,'selfidentifiedauthors',pathsave)
#self_study_track      = Annualized_Usage(nanohub_metrics_db,start_year,self_study_usernames,'unclassified',pathsave)
#simtotal_track        = Annualized_Usage(nanohub_metrics_db,start_year,allusers_usernames,'simtotal',pathsave)

#self_study_day_1_users_track        = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_1_users,'self_study_day_1_users',pathsave)
#self_study_day_2_7_users_track      = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_2_7_users,'self_study_day_2_7_users',pathsave)
#self_study_day_8_30_users_track     = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_8_30_users,'self_study_day_8_30_users',pathsave)
# self_study_day_31_90_users_track    = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_31_90_users,'self_study_day_31_90_users',pathsave)
# self_study_day_91_180_users_track   = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_91_180_users,'self_study_day_91_180_users',pathsave)
# self_study_day_181_365_users_track  = Annualized_Usage(nanohub_metrics_db,start_year,self_study_day_181_365_users,'self_study_day_181_365_users',pathsave)
# self_study_years_users_track        = Annualized_Usage(nanohub_metrics_db,start_year,self_study_years_users,'self_study_years_users',pathsave)

clustered_all_day_1_users_track        = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_1_users,      'clustered_all_day_1_users',pathsave)
clustered_all_day_2_7_users_track      = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_2_7_users,    'clustered_all_day_2_7_users',pathsave)
# clustered_all_day_8_30_users_track     = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_8_30_users,   'clustered_all_day_8_30_users',pathsave)
# clustered_all_day_31_90_users_track    = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_31_90_users,  'clustered_all_day_31_90_users',pathsave)
# clustered_all_day_91_180_users_track   = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_91_180_users, 'clustered_all_day_91_180_users',pathsave)
# clustered_all_day_181_365_users_track  = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_day_181_365_users,'clustered_all_day_181_365_users',pathsave)
# clustered_all_years_users_track        = Annualized_Usage(nanohub_metrics_db,start_year,clustered_all_years_users,      'clustered_all_years_users',pathsave)




2022-03-04 17:29:09.878608
end date: '2000-01-01'


OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'nan' in 'where clause'")
[SQL: select user from toolstart where user in ('0.yao.yuan', '007sribabu', '01joeeoj10', '01luiyi', '04ssteves', '0909hfn', '09petunia', '0osgxasfzu', '1071114545jeff', '10enricogiglio', '111icoolboy111', '11212121asd', '11jazzyc', '11pandeyd', '122guy122', '1234burhan', '1234qwer', '123wendyronquillo123', '12902791r', '12kj', '12tmdwn', '135goodguy', '13susant', '14.nt.mital.nanda', '14.nt.zalak.bhut', '14p3090', '14pwest', '14t2806h', '14tracolf', '15.jrendon.2b', '1501heo', '15105231236zyf', '15181518lxt', '15kochk', '16.nt.rajan.suhagiya', '1621150lexa', '16cirwin', '16d070026', '16smarty', '1754ayush', '1770341964zzw', '17w1008e', '17w4024c', '184005sasi', '18isalways18hhh', '1903setoo', '1991himani', '1991ivanzhang', '1993sonu01', '1994.vishaksunil', '1996jaypatel', '19af34966', '19denny79', '19mph004', '19sanjana92', '1heluckyrocketeer2', '1j2es3s4i5c6a7', '1pilo', '1steven1conger1

In [ ]:
print('clustered_track ',len(clustered_track))
print('clustered_m_track ', len(clustered_m_track))
print('clustered_x_track ',len(clustered_x_track))
print('clustered_mx_track ',len(clustered_mx_track))
print('researcher_track ',len(researcher_track))
print('self_identified_track ', len(self_identified_track))
print('self_study_track ',len(self_study_track))
print('simtotal_track ',len(simtotal_track))
print('all_months_list ',len(all_months_list))
